# Mallide põhjal näitelausete otsimine

In [28]:
#impordid
import pyconll

### Kasutame sisendina faili 'korpuselaused_verbiti_filtreeritud.ipynb' rakendamise tulemusel saadud faile 

In [29]:
#loeme verbide märgendusega failid sisse
def loe(verb):
    failitee = 'data/paringutulemused/' + verb + '.conllu'
    sisu = pyconll.iter_from_file(failitee)
    return sisu

### Paneb iga verbi esinemise otsesed alluvad õigel kujul listi

In [30]:
#paneme iga verbi instantsi jaoks tema alluvad kirja juhul, kui ta läbib imps ja imperi filtrid
def alluvad_fun(verb, laused):
    laused_pealikute_alluvatega = {} #key lause, value head_id_alluvad
    for lause in laused:
        head_id_alluvad = {} #key verbi id, value dict, mille key on alluva id, value sellele vastav lauseliige
        for sona in lause:
            
            #kui sõna on kaassõna
            if sona.deprel == 'case':
                pea_id = sona.head
                #kui kaassõna pea pea on meie verb
                if 'H_lemma' in lause[pea_id].deps.keys() and verb in lause[pea_id].deps['H_lemma']:
                    pea_pea_id = lause[pea_id].head
                    pea_dep = lause[pea_id].deprel
                    lauseliige = (pea_dep, sona.form, '', '')
                    if 'Case' in lause[pea_id].feats.keys():
                        pea_case = list(lause[pea_id].feats['Case'])[0]
                        lauseliige = (pea_dep, pea_case, sona.form, '')  
                    #lisame lauseliikme verbi alluvate hulka
                    if pea_pea_id not in head_id_alluvad.keys(): 
                        head_id_alluvad[pea_pea_id] = {pea_id: lauseliige}
                    else:
                        head_id_alluvad[pea_pea_id][pea_id] = lauseliige
                    
            
            #kui sõna ülemus on meie verb ja alluv meie otsitavate alluvate seas
            if ('H_lemma' in sona.deps.keys() and verb in sona.deps['H_lemma'] and 
                sona.deprel in ['nsubj', 'obj', 'xcomp', 'obl', 'ccomp', 'advmod', 'advcl', 'root', 'case']):
                #subjekt ja objekt
                if sona.deprel in ['nsubj', 'obj']:
                    lauseliige = (sona.deprel, '*', '', '')
                #obliikva ja xcomp
                elif sona.deprel in ['obl', 'xcomp'] and 'Case' in sona.feats.keys(): 
                    kaane = list(sona.feats['Case'])
                    lauseliige = (sona.deprel, kaane[0], '', '')
                    #lisame xcompile vormilõpu
                    if sona.deprel == 'xcomp' and kaane[0] == 'ill':
                        lauseliige = (sona.deprel, kaane[0], '', 'ma')
                elif sona.deprel in ['obl', 'xcomp'] and 'Case' not in sona.feats.keys():
                    lauseliige = (sona.deprel, '', '', '')
                #ccomp ja advmod
                elif sona.deprel in ['ccomp', 'advmod'] and 'Case' not in sona.feats.keys():
                    lauseliige = (sona.deprel, '<puudub>', '', '')
                #advcl
                elif sona.deprel == 'advcl' and 'VerbForm' in sona.feats.keys() and 'ger' in sona.feats['VerbForm']:
                    lauseliige = (sona.deprel, '<puudub>', '', 'des')
                #kõik muu
                else:
                    lauseliige = (sona.deprel, '', '', '')
                #lisame lauseliikme verbi alluvaks
                if sona.head not in head_id_alluvad.keys(): 
                    head_id_alluvad[sona.head] = {sona.id: lauseliige}
                elif sona.id not in head_id_alluvad[sona.head].keys():
                    head_id_alluvad[sona.head][sona.id] = lauseliige
        
        #lisame lause juurde verbi ja selle alluvad
        if bool(head_id_alluvad): 
            laused_pealikute_alluvatega[lause] = head_id_alluvad
    
    return laused_pealikute_alluvatega

### Kui mall esineb verbi otsestes alluvates, prindib lause ja alluvad

In [31]:
#leiame laused, kus malli liikmed esinevad alluvatena
def naitelaused(verb, mall):
    naited = []
    sisu = loe(verb)
    laused_pealikute_alluvatega = alluvad_fun(verb, sisu)
    for lause, pealikud in laused_pealikute_alluvatega.items():
        for pealik, alluvad in pealikud.items():
            #kui mall on alluvates
            if ((len(mall) == 1 and mall[0] in alluvad.values()) or 
                (len(mall) == 2 and mall[0] in alluvad.values() and mall[1] in alluvad.values()) or
                (len(mall) == 3 and mall[0] in alluvad.values() and mall[1] in alluvad.values() and mall[2] in alluvad.values())
                or (len(mall) == 4 and mall[0] in alluvad.values() and mall[1] in alluvad.values() 
                   and mall[2] in alluvad.values() and mall[3] in alluvad.values())):
                print(lause.text)
                print('verb: ', lause[pealik].form, ' kohal ', pealik)
                for alluva_id, alluv in alluvad.items():
                    print(lause[alluva_id].form, ': ', alluv)
                print()
                naited.append(lause.conll())
    return naited

### Kui kõik verbi otsesed alluvad on täpselt samad mis malli omad, prindib lause ja alluvad

In [32]:
#leiame laused, mis jälgivad täpselt malli
def naitelaused_tapne(verb, mall):
    naited = []
    sisu = loe(verb)
    laused_pealikute_alluvatega = alluvad_fun(verb, sisu)
    for lause, pealikud in laused_pealikute_alluvatega.items():
        for pealik, alluvad in pealikud.items():
            #kui mall on alluvates
            if mall == list(alluvad.values()):
                print(lause.text)
                print('verb: ', lause[pealik].form, ' kohal ', pealik)
                for alluva_id, alluv in alluvad.items():
                    print(lause[alluva_id].form, ': ', alluv)
                print()
                naited.append(lause.conll())
    return naited

### Sisesta verb ja mall ja prindib näited

In [33]:
verb = 'tõstma'
lausemall = [('obl', 'term', '', '')]

naited = naitelaused(verb, lausemall)

Tehnoloogiaaktsiad jätkasid teisipäeval New Yorgis kukkumist , raha voolas sealt " vana majanduse " aktsiatesse , mis tõstis Dow Jonesi indeksi viimase kolme kuu kõrgeima tasemeni .
verb:  tõstis  kohal  18
mis :  ('nsubj', '*', '', '')
indeksi :  ('obj', '*', '', '')
tasemeni :  ('obl', 'term', '', '')

Rahandusminister Taavi Veskimäe allkirjaga eelnõu tõstab maamaksumäära alampiiri seniselt 0,1 protsendilt ühe protsendini maa väärtusest .
verb:  tõstab  kohal  6
eelnõu :  ('obl', 'gen', '', '')
alampiiri :  ('obj', '*', '', '')
seniselt :  ('advmod', '<puudub>', '', '')
protsendilt :  ('obl', 'abl', '', '')
protsendini :  ('obl', 'term', '', '')

Võru Juust täitis aasta alguses prognoositud kasumi üheksa kuuga ning tõstis seejärel kasumiprognoosi 17 miljoni kroonini .
verb:  tõstis  kohal  11
seejärel :  ('advmod', '<puudub>', '', '')
kasumiprognoosi :  ('obj', '*', '', '')
kroonini :  ('obl', 'term', '', '')

Inteli Pentium 4 protsessor muutus jälle pisut kiiremaks , firma tõstis ki